In [1]:
%pylab inline
%gui qt

Populating the interactive namespace from numpy and matplotlib


In [2]:
from spiketag.utils import EventEmitter
from spiketag.base import CLU

In [3]:
class status_manager(EventEmitter):
    def __init__(self):
        super(status_manager, self).__init__()
        self.reporters = {}
        self._event_reg_enable = True
    
    def append(self, state_reporter):
        self.reporters[state_reporter._id] = state_reporter
        
        @state_reporter.connect
        def on_report(state):
#             print(state+' from group '+ str(state_reporter._id))
              state_reporter.state = state
              self.emit('update', state=self.state_list, nclu=self.nclu_list)

            
    def __getitem__(self, i):
        return self.reporters[i]
    
    def __setitem__(self, i, state_str):
        self.reporters[i].state = state_str
    
    def __repr__(self):
        _str = ''
        for _, _reporter in self.reporters.items():
            _str = _str + str(_reporter._id) + ":" + str(_reporter.state) + '\n'
        return _str
    
    def reset(self):
        for reports in self.reporters.values():
            reports._state = 'IDLE'
        self.emit('update', state=self.state_list, nclu=self.nclu_list)
    
    @property
    def state_list(self):
        return [troy.s.index(troy.state) for troy in self.reporters.values()]
    
    @property
    def ngroup(self):
        return len(self.reporters.keys())
    
    @property
    def nclu_list(self):
        return [troy.nclu for troy in self.reporters.values()]

In [6]:
from vispy import app, scene, visuals
from vispy.util import keys
import numpy as np


class cluster_view(scene.SceneCanvas):
    def __init__(self):
        scene.SceneCanvas.__init__(self, keys=None, title='clusters overview')
        self.unfreeze()
        self.view = self.central_widget.add_view()
        self.view.camera = 'panzoom'    
        # every group (grp_marker) has a clustering result each cluster will has a (clu_marker)
        # every group has its status: finish, unfinished
        # every clu has its status: spike counts, high quality, low quality, information bits
        self.grp_marker  = scene.visuals.Markers(parent=self.view.scene)
        self.nclu_text = scene.visuals.Text(parent=self.view.scene)
        self.event = EventEmitter() 


    def set_data(self, clu_manager, selected_group_id, size=25):
        '''
        group_No is a scala number #grp
        nclu_list is a list with length = group_No
        '''
        
        self.clu_manager = clu_manager
        
        self.group_No = self.clu_manager.ngroup
        self.nclu_list = np.array(self.clu_manager.nclu_list)
        self.sorting_status = np.array(self.clu_manager.state_list)
        self.nspks_list = None
        self._size = size

        self.xmin = -0.02
        self.xmax =  0.04
        grp_x_pos = np.zeros((self.group_No,))
        grp_y_pos = np.arange(self.group_No)
        self.grp_pos = np.vstack((grp_x_pos, grp_y_pos)).T
        self.nclu_text_pos = np.vstack((grp_x_pos+0.02, grp_y_pos)).T
        
        self.current_group, self.selected_group_id = selected_group_id, selected_group_id

        self.color = self.generate_color(self.sorting_status, self.nspks_list, self.selected_group_id) 
        self.ecolor = np.zeros_like(self.color)
        self.ecolor[self.current_group] = np.array([0,1,0,1])

        self.grp_marker.set_data(self.grp_pos, symbol='square', 
                                 face_color=self.color, edge_color=self.ecolor, edge_width=4, size=size)
        
        self.nclu_text.text = [str(i) for i in self.nclu_list]
        self.nclu_text.pos  = self.nclu_text_pos
        self.nclu_text.color = 'g'
        self.nclu_text.font_size = size*0.50

        self.view.camera.set_range(x=[self.xmin, self.xmax])
        # self.view.camera.interactive = False
        
        if self.clu_manager._event_reg_enable:
            self.event_register()
        
        
    def event_register(self):
        @self.clu_manager.connect
        def on_update(state, nclu):
        #     print(state)
        #     print(nclu)
            self.refresh()
        self.clu_manager._event_reg_enable = not self.clu_manager._event_reg_enable


    def generate_color(self, sorting_status, nspks_list, selected_group_id):
        self.color = np.ones((self.group_No, 4)) * 0.5
        self.color[sorting_status==0] = np.array([1,1,1, .3]) # IDLE
        self.color[sorting_status==1] = np.array([1,0,0, 1.]) # BUSY
        self.color[sorting_status==2] = np.array([0,1,0, .7]) # READY
        self.color[sorting_status==3] = np.array([1,1,0, .8]) # READY
#         self.color[selected_group_id] = (np.array([1,1,1, 1]) + self.color[selected_group_id])/2  # current selected
        if nspks_list is not None:
            self.transparency = np.array(nspks_list)/np.array(nspks_list).max()
            self.color[:, -1] = self.transparency
        return self.color 


    def on_key_press(self, e):
        if e.text == 'r':
            self.view.camera.set_range(x=[self.xmin, self.xmax])
        if e.text == 'k':
            self.moveto(self.next_group)
        if e.text == 'j':
            self.moveto(self.previous_group)
        if e.text == 'd':
            self.set_cluster_done(self.current_group)
#             self.moveto(self.next_group)
        if e.text == 'o':
            self.select(self.current_group)

    @property
    def cpu_ready_list(self):
        return np.where(self.sorting_status==1)[0]

    def set_cluster_ready(self, grp_id):
        self.sorting_status[grp_id] = 1
        self.refresh()

    def set_cluster_done(self, grp_id):
        self.clu_manager[grp_id] = 'DONE'
        self.refresh()
        
#     def update(self):
#         self.clu_manager.emit('update', state=self.clu_manager.state_list, nclu=self.clu_manager.nclu_list)

    def refresh(self):
        self.set_data(clu_manager=self.clu_manager, selected_group_id=self.current_group, size=self._size)


    @property
    def previous_group(self):
        if self.current_group>0:
            self._previous_group = self.current_group - 1
            return self._previous_group
        else:
            self._previous_group = 0
            return self._previous_group 


    @property
    def next_group(self):
        if self.current_group<self.group_No-1:
            self._next_group = self.current_group + 1
            return self._next_group
        else:
            return self._next_group 


    def moveto(self, group_id):
        self.current_group = group_id
        self.set_data(self.clu_manager, self.current_group, self._size) 


    def select(self, group_id):
        # if self.sorting_status[group_id] != 0:
        self.event.emit('select', group_id=self.current_group)
        # else:
            # print('unable to select busy cpu {}'.format(self.current_group)) 


    def run(self):
        self.show()
        self.app.run()

In [7]:
cluview = cluster_view()

In [8]:
clu_manager = status_manager()

In [9]:
for i in range(10):
    clu = CLU(np.arange(i+1))
    clu._id = i
    clu_manager.append(clu)

In [10]:
clu_manager.state_list

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [11]:
clu_manager.nclu_list

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [12]:
# disconnect if `update` function has something wrong
# clu_manager._callbacks.pop('update')

In [13]:
cluview.show()

In [14]:
cluview.set_data(clu_manager, selected_group_id=0)

In [15]:
clu_manager[8] = 'READY'
clu_manager[6] = 'BUSY'
clu_manager[3] = 'DONE'

In [16]:
cluview.refresh()

In [13]:
clu_manager.reset()

In [14]:
_clu = clu_manager[8]

In [27]:
### now just need to make sure where after fill method to put state report
clu_manager[8].fill(np.arange(9))
# clu_manager[8].emit('report', state='BUSY')

8

In [17]:
clu_manager[4].emit('report', state='BUSY')

[None]

In [26]:
clu_manager[8].nclu

9

In [27]:
clu_manager[8].npts

9

In [31]:
clu_manager

0:IDLE
1:IDLE
2:IDLE
3:DONE
4:DONE
5:IDLE
6:BUSY
7:DONE
8:READY
9:IDLE